In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import segment_speed_utils

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [3]:
## parameters cell
itp_id = 182

In [4]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt

In [5]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)

In [6]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [7]:
speedmap_segs = gpd.read_parquet(path) #  aggregated

In [8]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [9]:
#  TODO to utils
def time_period_and_arrowize(gdf, time_period):
    
    gdf = gdf >> filter(_.time_of_day == time_period)
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1, frequency_col='trips_hr_sch')

    gdf = gdf >> arrange(_.trips_hr_sch)

    return gdf

In [10]:
peak_test = time_period_and_arrowize(speedmap_segs, 'AM Peak')

In [11]:
cmap = shared_utils.rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.ACCESS_SPEEDMAP_LEGEND_URL

In [12]:
peak_test >> head(2)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,direction_id,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch,route_short_name
10501,0666caf3ec1ecc96b74f4477ee4bc939,fedc605c108af558e4e545806741ea0f,510358_JUNE24,51-13183,0.0,283__8824,Avalon / Imperial__Avalon / 111th,283-8824-1,AM Peak,12.34,...,12.34,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"POLYGON ((160449.331 -452365.708, 160434.047 -...",1,0.333333,51
10502,0666caf3ec1ecc96b74f4477ee4bc939,fedc605c108af558e4e545806741ea0f,510358_JUNE24,51-13183,0.0,8824__8823,Avalon / 111th__Avalon / 110th,8824-8823-1,AM Peak,14.66,...,14.66,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"POLYGON ((160443.819 -452040.084, 160428.472 -...",1,0.333333,51


## export map

In [63]:
time_of_day_lower = time_of_day.lower().replace(' ', '_')

In [65]:
#  TODO clone render method from RtFilterMapper, to utils
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/182_pm_peak_new.geojson.gz


In [88]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs PM Peak 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/182_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (34.0539357609537, -118.29997523003668),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend_color_access.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIFBNIFBlYWsgMjAyNC0xMC0xNiIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy8xODJfbmV3Lmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNSwgInRvb2x0aXBfc3BlZWRfa2V5IjogInAyMF9tcGgifSwgInR5cGUiOiAic3BlZWRtYXAifV0sICJsYXRfbG9uIjogWzM0LjA1MzkzNTc2MDk1MzcsIC0xMTguMjk5OTc1MjMwMDM2NjhdLCAi

In [ ]:
## TODO get a better chart -- p20 - 50 - 80 seg speed?
# try:
#     rt_day.set_filter(start_time='06:00', end_time='22:00')
#     display(Markdown(f'Median Trip Speed by Hour of Day{rt_day.filter_formatted}'))

#     rt_day.chart_speeds(no_title=True);
# except:
#     display(Markdown(f'Chart not available for this agency'))
#     pass